In [1]:
import glob
import re

import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import scienceplots

plt.style.use("science")

In [6]:
activations = ["sigmoid", "tanh", "relu", "JNB5", "JNB6"]
DATA = []
SCORES = []
for activation in activations:
    files = sorted(glob.glob("data/*%s*.txt" % activation))
    alg1 = []
    alg2 = []
    for file in files:
        history1, score1, history2, score2 = list(
            map(lambda t: eval(str.strip(t)), open(file).readlines())
        )
        data = []
        score = []
        for history, score_ in [(history1, score1), (history2, score2)]:
            d = (
                history["loss"],
                history["val_loss"],
                history["mse"],
                history["val_mse"],
                history["psnr"],
                history["val_psnr"],
                history["ssim"],
                history["val_ssim"],
            )
            data.append(d)
            score.append(score_)

        alg1.append(data)
        alg2.append(score)
    DATA.append(alg1)
    SCORES.append(alg2)
DATA = np.array(DATA)
SCORES = np.array(SCORES)
DATA.shape, SCORES.shape

((5, 5, 2, 8, 20), (5, 5, 2, 4))

In [7]:
x = list(range(1, 1 + 20))
mapping = {
    "JNB2": "JNB(2)",
    "JNB3": "JNB(3)",
    "JNB4": "JNB(4)",
    "JNB5": "JNB(5)",
    "JNB6": "JNB(6)",
    "RELU": "ReLU",
    "SIGMOID": "Sigmoid",
    "TANH": "Tanh",
}

names = []
for i in range(0, len(activations)):
    names.append(mapping[activations[i].upper()])

In [8]:
mean = SCORES.mean(axis=1)
std = SCORES.std(axis=1)

In [9]:
mean.shape

(5, 2, 4)

In [10]:
columns = ["Loss", "MSE", "PSNR", "SSIM"]
df = pd.DataFrame(index=names, columns=columns)
alg = 0

# Populate DataFrame with mean +- std
for i in range(mean.shape[0]):
    for j in range(mean.shape[2]):
        df.at[
            names[i], columns[j]
        ] = f"${mean[i, alg, j]:.3f} \pm {std[i, alg, j]:.3f}$"

df

,Loss,MSE,PSNR,SSIM
Sigmoid,$0.298 \pm 0.011$,$0.018 \pm 0.003$,$18.026 \pm 0.709$,$0.579 \pm 0.043$
Tanh,$0.295 \pm 0.007$,$0.017 \pm 0.002$,$18.078 \pm 0.545$,$0.583 \pm 0.035$
ReLU,$0.294 \pm 0.002$,$0.017 \pm 0.001$,$18.124 \pm 0.227$,$0.577 \pm 0.013$
JNB(5),$0.283 \pm 0.002$,$0.013 \pm 0.001$,$19.205 \pm 0.300$,$0.647 \pm 0.015$
JNB(6),$0.284 \pm 0.003$,$0.013 \pm 0.001$,$19.171 \pm 0.337$,$0.648 \pm 0.019$


In [11]:
columns = ["Loss", "MSE", "PSNR", "SSIM"]
df2 = pd.DataFrame(index=names, columns=columns)
alg = 1
# Populate DataFrame with mean +- std
for i in range(mean.shape[0]):
    for j in range(mean.shape[2]):
        df2.at[
            names[i], columns[j]
        ] = f"${mean[i, alg, j]:.2f} \pm {std[i, alg, j]:.2f}$"
df2
# print(df.reset_index(drop=True))

,Loss,MSE,PSNR,SSIM
Sigmoid,$0.30 \pm 0.00$,$0.02 \pm 0.00$,$17.81 \pm 0.35$,$0.56 \pm 0.02$
Tanh,$0.30 \pm 0.00$,$0.02 \pm 0.00$,$17.83 \pm 0.26$,$0.56 \pm 0.02$
ReLU,$0.30 \pm 0.00$,$0.02 \pm 0.00$,$17.54 \pm 0.19$,$0.54 \pm 0.01$
JNB(5),$0.29 \pm 0.00$,$0.02 \pm 0.00$,$18.16 \pm 0.31$,$0.58 \pm 0.02$
JNB(6),$0.29 \pm 0.00$,$0.02 \pm 0.00$,$18.21 \pm 0.17$,$0.59 \pm 0.01$
